In [1]:
import pandas as pd
from torch import np # Torch wrapper for Numpy

import os
from PIL import Image

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import Sampler, SequentialSampler, SubsetRandomSampler
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

import torchvision
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
IMG_PATH = 'train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = 'train_label.csv'

TEST_PATH = 'test-jpg/'
TEST_DATA = 'sample_submission_v2.csv'

NUM_CLASSES = 17
NUM_TRAIN = 37479
NUM_VAL = 3000

In [3]:
class KaggleAmazonDataset(Dataset):
    """Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None, split = 'train'):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['image_name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['image_name']
        self.y_train = None
        if split == 'train':
            self.y_train = self.mlb.fit_transform(tmp_df['tags'].str.split()).astype(np.float32)

    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        if self.y_train is not None:
            label = torch.from_numpy(self.y_train[index])
            return img, label
        else:
            return img

    def __len__(self):
        return len(self.X_train.index)

In [4]:
import random

def randomTranspose(x):
    if random.random() < 0.5:
        x = x.transpose(1,2)
    return x

transformations = transforms.Compose([transforms.Scale(256),transforms.RandomHorizontalFlip(), \
                                      transforms.ToTensor(), transforms.Lambda(lambda x: randomTranspose(x)) ])
#, transforms.Lambda(lambda x: randomTranspose(x.numpy()))
dset_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)

In [5]:
# class ChunkSampler(Sampler):
#     """Samples elements sequentially from some offset. 
#     Arguments:
#         num_samples: # of desired datapoints
#         start: offset where we should start selecting from
#     """
#     def __init__(self, num_samples, start = 0):
#         self.num_samples = num_samples
#         self.start = start

#     def __iter__(self):
#         return iter(range(self.start, self.start + self.num_samples))

#     def __len__(self):
#         return self.num_samples
    

# loader_train = DataLoader(dset_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))

# loader_val = DataLoader(dset_train, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

In [6]:
# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor

In [7]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(2304, 256)
#         self.fc2 = nn.Linear(256, 17)

#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(x.size(0), -1) # Flatten layer
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.sigmoid(x)

# # model = Net() # On CPU
# model = Net().cuda() # On GPU

class MyModel(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.last_layer = nn.Sigmoid()

    def forward(self, x):
        return self.last_layer(self.pretrained_model(x))

pretrained_model = torchvision.models.resnet18(pretrained=True)
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, NUM_CLASSES)
# print(pretrained_model)
model = MyModel(pretrained_model)

In [8]:
model.type(dtype)
for param in model.parameters():
    param.requires_grad = False
for param in model.pretrained_model.fc.parameters():
    param.requires_grad = True

In [9]:
optimizer = torch.optim.Adam(model.pretrained_model.fc.parameters(), lr=1e-3)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [10]:
train_loader = DataLoader(dset_train,
                          batch_size=32,
                          shuffle = True,
                          drop_last = True,
                          num_workers=1, # 1 for CUDA
#                           sampler = ChunkSampler(NUM_TRAIN, 0),
#                           sampler = SubsetRandomSampler(range(NUM_TRAIN)),
                          pin_memory=True # CUDA only
                         )

# val_loader = DataLoader(dset_train,
#                           batch_size=32,
#                           drop_last = True,
#                           num_workers=1, # 1 for CUDA
# #                           sampler = ChunkSampler(NUM_VAL, NUM_TRAIN),
#                           sampler = SubsetRandomSampler(range(NUM_TRAIN, NUM_TRAIN + NUM_VAL)),
#                           pin_memory=True # CUDA only
#                          )


In [11]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        
#     model.eval()
#     val_num = 0
#     val_score = 0
#     val_loss = 0
    
#     for batch_idx, (data, target) in enumerate(val_loader):
#         data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
#         data, target = Variable(data), Variable(target)
#         output = model(data)
#         loss = F.binary_cross_entropy(output, target)
        
#         val_num += len(data)
#         val_loss += loss
        
#     val_loss /= val_num
#     print("Val loss: {:.6f}".format(val_loss))

        
        
    

In [12]:
for epoch in range(1, 2):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.828939
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.207056
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.152558
Train Epoch: 1 [9600/40479 (24%)]	Loss: 0.202723
Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.161529
Train Epoch: 1 [16000/40479 (40%)]	Loss: 0.123555
Train Epoch: 1 [19200/40479 (47%)]	Loss: 0.158530
Train Epoch: 1 [22400/40479 (55%)]	Loss: 0.114603
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.145887
Train Epoch: 1 [28800/40479 (71%)]	Loss: 0.131140
Train Epoch: 1 [32000/40479 (79%)]	Loss: 0.146387
Train Epoch: 1 [35200/40479 (87%)]	Loss: 0.129689
Train Epoch: 1 [38400/40479 (95%)]	Loss: 0.115407


In [13]:
for epoch in range(1, 5):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.123179
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.086397
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.141070
Train Epoch: 1 [9600/40479 (24%)]	Loss: 0.140612
Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.209979
Train Epoch: 1 [16000/40479 (40%)]	Loss: 0.148314
Train Epoch: 1 [19200/40479 (47%)]	Loss: 0.156550
Train Epoch: 1 [22400/40479 (55%)]	Loss: 0.154521
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.111946
Train Epoch: 1 [28800/40479 (71%)]	Loss: 0.103581
Train Epoch: 1 [32000/40479 (79%)]	Loss: 0.154220
Train Epoch: 1 [35200/40479 (87%)]	Loss: 0.112811
Train Epoch: 1 [38400/40479 (95%)]	Loss: 0.109570
Train Epoch: 2 [0/40479 (0%)]	Loss: 0.107110
Train Epoch: 2 [3200/40479 (8%)]	Loss: 0.147493
Train Epoch: 2 [6400/40479 (16%)]	Loss: 0.153570
Train Epoch: 2 [9600/40479 (24%)]	Loss: 0.156646
Train Epoch: 2 [12800/40479 (32%)]	Loss: 0.141089
Train Epoch: 2 [16000/40479 (40%)]	Loss: 0.143304
Train Epoch: 2 [19200/40479 (47%)]	Loss: 0.186163
Train Epoch: 2 [22

In [14]:
for epoch in range(1, 3):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.171668
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.149592
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.120410
Train Epoch: 1 [9600/40479 (24%)]	Loss: 0.175223
Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.097989
Train Epoch: 1 [16000/40479 (40%)]	Loss: 0.155856
Train Epoch: 1 [19200/40479 (47%)]	Loss: 0.135132
Train Epoch: 1 [22400/40479 (55%)]	Loss: 0.192863
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.129870
Train Epoch: 1 [28800/40479 (71%)]	Loss: 0.115218
Train Epoch: 1 [32000/40479 (79%)]	Loss: 0.135781
Train Epoch: 1 [35200/40479 (87%)]	Loss: 0.147131
Train Epoch: 1 [38400/40479 (95%)]	Loss: 0.184221
Train Epoch: 2 [0/40479 (0%)]	Loss: 0.136432
Train Epoch: 2 [3200/40479 (8%)]	Loss: 0.138501
Train Epoch: 2 [6400/40479 (16%)]	Loss: 0.146924
Train Epoch: 2 [9600/40479 (24%)]	Loss: 0.133526
Train Epoch: 2 [12800/40479 (32%)]	Loss: 0.159796
Train Epoch: 2 [16000/40479 (40%)]	Loss: 0.182151
Train Epoch: 2 [19200/40479 (47%)]	Loss: 0.149135
Train Epoch: 2 [22

In [15]:
optimizer = torch.optim.Adam(model.pretrained_model.fc.parameters(), lr=1e-4)

for epoch in range(1, 2):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.120348
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.131251
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.117261
Train Epoch: 1 [9600/40479 (24%)]	Loss: 0.140783
Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.105033
Train Epoch: 1 [16000/40479 (40%)]	Loss: 0.119596
Train Epoch: 1 [19200/40479 (47%)]	Loss: 0.136888
Train Epoch: 1 [22400/40479 (55%)]	Loss: 0.145551
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.130579
Train Epoch: 1 [28800/40479 (71%)]	Loss: 0.156442
Train Epoch: 1 [32000/40479 (79%)]	Loss: 0.149531
Train Epoch: 1 [35200/40479 (87%)]	Loss: 0.143198
Train Epoch: 1 [38400/40479 (95%)]	Loss: 0.163327


In [16]:
for epoch in range(1, 3):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.151192
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.138406
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.137685
Train Epoch: 1 [9600/40479 (24%)]	Loss: 0.116214
Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.137760
Train Epoch: 1 [16000/40479 (40%)]	Loss: 0.102601
Train Epoch: 1 [19200/40479 (47%)]	Loss: 0.151126
Train Epoch: 1 [22400/40479 (55%)]	Loss: 0.110171
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.100256
Train Epoch: 1 [28800/40479 (71%)]	Loss: 0.107878
Train Epoch: 1 [32000/40479 (79%)]	Loss: 0.162737
Train Epoch: 1 [35200/40479 (87%)]	Loss: 0.113197
Train Epoch: 1 [38400/40479 (95%)]	Loss: 0.115904
Train Epoch: 2 [0/40479 (0%)]	Loss: 0.093670
Train Epoch: 2 [3200/40479 (8%)]	Loss: 0.123251
Train Epoch: 2 [6400/40479 (16%)]	Loss: 0.126515
Train Epoch: 2 [9600/40479 (24%)]	Loss: 0.150035
Train Epoch: 2 [12800/40479 (32%)]	Loss: 0.125778
Train Epoch: 2 [16000/40479 (40%)]	Loss: 0.158569
Train Epoch: 2 [19200/40479 (47%)]	Loss: 0.145509
Train Epoch: 2 [22

In [17]:
for param in model.parameters():
    param.requires_grad = True

# Construct a new Optimizer that will update all model parameters. Note the
# small learning rate.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
for epoch in range(1, 2):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.127204
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.136197
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.082399
Train Epoch: 1 [9600/40479 (24%)]	Loss: 0.110119
Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.148342
Train Epoch: 1 [16000/40479 (40%)]	Loss: 0.115964
Train Epoch: 1 [19200/40479 (47%)]	Loss: 0.132456
Train Epoch: 1 [22400/40479 (55%)]	Loss: 0.170865
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.095612
Train Epoch: 1 [28800/40479 (71%)]	Loss: 0.107046
Train Epoch: 1 [32000/40479 (79%)]	Loss: 0.085151
Train Epoch: 1 [35200/40479 (87%)]	Loss: 0.077145
Train Epoch: 1 [38400/40479 (95%)]	Loss: 0.115007


In [ ]:
for epoch in range(1, 5):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.098843
Train Epoch: 1 [3200/40479 (8%)]	Loss: 0.063813
Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.073177

In [ ]:
for epoch in range(1, 4):
    train(epoch)

In [30]:
from utils import optimise_f2_thresholds
def find_best_thresholds():
    train_loader = DataLoader(dset_train,
                          batch_size=32,
                          drop_last = True,
                          num_workers=1, # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )
    
    model.eval()
    train_num = 0
    predictions = np.zeros((dset_train.__len__(), NUM_CLASSES ))
    true_labels = np.zeros((dset_train.__len__(), NUM_CLASSES ))
    for batch_idx, (data, target) in enumerate(train_loader, 0):
        data = data.cuda(async=True) # On GPU
        data = Variable(data)
        output = model(data)
        
        batch_size = len(data)
        train_num += batch_size
        
        start = train_num - batch_size
        end = train_num
        predictions[start:end] = output.data.cpu().numpy().reshape(-1,NUM_CLASSES)
        true_labels[start:end] = target.cpu().numpy().reshape(-1, NUM_CLASSES)
        
    thresh = optimise_f2_thresholds(true_labels, predictions)
    
    return thresh
        
        
    

In [31]:
thresh = find_best_thresholds()

0 0.19 0.935949303651
1 0.24 0.93596613717
2 0.13 0.93605152513
3 0.34 0.936141211012
4 0.13 0.936164196754
5 0.18 0.936219753937
6 0.06 0.936920077516
7 0.18 0.936925966846
8 0.18 0.937084065383
9 0.14 0.937139906395
10 0.25 0.937310623385
11 0.17 0.937341140884
12 0.29 0.937517349047
13 0.18 0.937567371028
14 0.2 0.937567371028
15 0.11 0.937640218572
16 0.16 0.937712715456


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [32]:
dset_test = KaggleAmazonDataset(TEST_DATA,TEST_PATH,IMG_EXT, transformations, split = 'test')

In [33]:
test_loader = DataLoader(dset_test,
                          batch_size = 128,
                          drop_last = False,
                          sampler = SequentialSampler(dset_test),
                          num_workers=4 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

61191

In [34]:
def predict(test_loader):
    model.eval()
    test_num = 0
    predictions = np.zeros((dset_test.__len__(), NUM_CLASSES ))
    for batch_idx, data in enumerate(test_loader, 0):
        data = data.cuda(async=True) # On GPU
        data = Variable(data)
        output = model(data)
        
        batch_size = len(data)
        test_num += batch_size
        
        start = test_num - batch_size
        end = test_num
        predictions[start:end] = output.data.cpu().numpy().reshape(-1,NUM_CLASSES)
        print(batch_idx)
        
    return predictions

In [35]:
preds = predict(test_loader)

print(preds.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [36]:
# thresh = 0.235

df_test = pd.read_csv('sample_submission_v2.csv')

outs = dset_train.mlb.inverse_transform(preds > thresh)
labelnames = [' '.join(lb) for lb in outs]


In [37]:

subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = labelnames
subm.to_csv('submission_resnet18_4.csv', index=False)

In [38]:
subm

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,agriculture clear cultivation primary
4,test_4,cloudy partly_cloudy primary
5,test_5,clear primary
6,test_6,agriculture clear cultivation habitation primary
7,test_7,clear habitation primary road water
8,test_8,clear primary
9,test_9,agriculture clear haze primary
